#### This assignment may be worked individually or in pairs. Enter your name/names here:
    

In [267]:
#names here
# Matthew Jagen

# Assignment 1: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy (Accumulative label for the Messidor classes 1, 2, 3), 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions for your convenience. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5.
- For binary features [0/1] the threshold value will be 0.5. All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [268]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import bisect
from math import log2
from random import shuffle

In [269]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [270]:
def get_data(filename):
    data = []
#     your code goes here
    dframe = pd.read_csv(filename, names=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19"])
    for i in range(len(dframe)):
        row = dframe.iloc[i]
        dpoint = DataPoint(row.name, row.values.tolist())
        data.append(dpoint)
        
    return data

# print(get_data("messidor_features.txt")[0])

In [271]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [272]:
def make_prediction(tree_root, data_point):
#     your code goes here
    if tree_root.is_leaf:
        return tree_root.prediction
    val = data_point.features[tree_root.feature_idx]
    if val < tree_root.thresh_val:
        return make_prediction(tree_root.left_child, data_point)
    if tree_root.right_child:
        return make_prediction(tree_root.right_child, data_point)
    return "prediction error"

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [273]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for point in data:
        val = point.features[feature_idx]
        if val < threshold:
            left_split.append(point)
        else:
            right_split.append(point)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [274]:
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    class_label_idx = 19
    num_signs = 0
    num_no_signs = 0
    
    for point in data:
        if point.features[class_label_idx] == 0:
            num_no_signs += 1
        else:
            num_signs += 1
    total = num_signs + num_no_signs
        
    #check for 0s
    if total == 0: #strange case
        return 0.0
    signs_entropy = 0.0
    if num_signs > 0: #ensure no 0s inside the log
        signs_entropy = -(num_signs/total)*log2(num_signs/total)
    no_signs_entropy = 0.0
    if num_no_signs > 0: #ensure no 0s inside the log
        no_signs_entropy = -(num_no_signs/total)*log2(num_no_signs/total)
        
    entropy = signs_entropy + no_signs_entropy
    
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [275]:
def find_all_thresholds(data, feature_idx):
    values = []
    thresholds = []
    for point in data:
        val = point.features[feature_idx]
        bisect.insort(values, val) #inserts values in increasing order
    for i in range(len(values)-1):
        if values[i] != values[i+1]:
            threshold = (values[i+1] - values[i])/2.0 + values[i]
            thresholds.append(threshold)
    return thresholds
    

def calculate_split_impurity(left_split, right_split):
    if len(left_split) == 0:
        return calc_entropy(right_split)
    if len(right_split) == 0:
        return calc_entropy(left_split)
    split_impurity = 0.0
    total_num = 0.0
    left_num = len(left_split)
    left_impurity = calc_entropy(left_split)
    right_num = len(right_split)
    right_impurity = calc_entropy(right_split)
    
    total_num = left_num + right_num
    split_impurity = (left_num/total_num)*(left_impurity) + (right_num/total_num)*(right_impurity)
    
    return split_impurity
    
# Calculates the gain of a split given impurity values
def calculate_gain(impurity_parent, impurity_split):
    return impurity_parent - impurity_split
    
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
    parent_impurity = calc_entropy(data)
    thresholds = find_all_thresholds(data, feature_idx)
    for threshold in thresholds:
        split_data = split_dataset(data, feature_idx, threshold)
        split_impurity = calculate_split_impurity(split_data[0], split_data[1])
        gain = calculate_gain(parent_impurity, split_impurity)
        if gain > best_info_gain:
            best_info_gain = gain
            best_thresh = threshold
        
    return (best_info_gain, best_thresh)

# data = get_data("messidor_features.txt")
# split_data = split_dataset(data, 6, 15)
# print(find_all_thresholds(split_data[1], 6))
# split_impurity = calculate_split_impurity(split_data[0], split_data[1])
# parent_impurity = calc_entropy(data)
# print(parent_impurity)
# print(split_impurity)
# print(calculate_gain(parent_impurity, split_impurity))
# print(calc_best_threshold(data, 6))


Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [276]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    best_gain = 0.0
    for i in range(len(data[0].features) - 1): #last feature is the class label
        feature_gain, feature_thresh = calc_best_threshold(data, i)
        if feature_gain > best_gain:
            best_gain = feature_gain
            best_thresh = feature_thresh
            best_feature = i
    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [277]:
def create_leaf_node(data):
#     your code goes here
    leaf = TreeNode()
    leaf.is_leaf = True
    class_label_idx = 19
    num_signs = 0
    num_no_signs = 0
    
    for point in data:
        if point.features[class_label_idx] == 0:
            num_no_signs += 1
        else:
            num_signs += 1
    if num_no_signs > num_signs:
        leaf.prediction = 0
    else:
        leaf.prediction = 1
    
    return leaf

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [278]:
def create_decision_tree(data, max_levels):
#     your code goes here
    if max_levels == 1:
        return create_leaf_node(data)
    if calc_entropy(data) == 0:
        return create_leaf_node(data)
    
    node = TreeNode()
    node.is_leaf = False
    
    feature, thresh = identify_best_split(data)
    node.feature_idx = feature
    node.thresh_val = thresh
    left_data, right_data = split_dataset(data, feature, thresh)
    
    node.left_child = create_decision_tree(left_data, max_levels-1)
    node.right_child = create_decision_tree(right_data, max_levels-1)
    
    return node

# data = get_data("messidor_features.txt")
# tree = create_decision_tree(data, 5)
# tree.printTree()

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [279]:
def calc_accuracy(tree_root, data):
#     your code goes here
    if len(data) == 0:
        return 0.0

    class_label_idx = 19
    num_correct = 0.0
    
    for point in data:
        prediction = make_prediction(tree_root, point)
        if prediction == point.features[class_label_idx]:
            num_correct += 1.0
        
    return num_correct/len(data)

# data = get_data("messidor_features.txt")
# tree = create_decision_tree(data, 5)
# print(calc_accuracy(tree, data))

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [280]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")
max_levels = 10
num_folds = 5
accuracies = []

# Cross-Validation loop (iterate 5 times)
for i in range(num_folds):
    # partition data into train_set and test_set
    partition_start_idx = int((i)*(len(d)/num_folds))
    partition_end_idx = int((i+1)*(len(d)/num_folds))
    test_set = d[partition_start_idx:partition_end_idx]
    train_set_first_half = d[:partition_start_idx]
    train_set_second_half = d[partition_end_idx:]
    train_set = train_set_first_half + train_set_second_half
    
    print ('\nTraining set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the decision tree
    start = time.time()
    tree = create_decision_tree(train_set, max_levels)
    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    accuracies.append(accuracy)
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    #t.printTree()

# print the accuracy of the whole model using the final estimate of generalization error
final_accuracy = 0.0
for a in accuracies:
    final_accuracy += a
final_accuracy /= len(accuracies)

print ('\nThe accuracy of the model is ', str(final_accuracy * 100.0))


Training set size: 920
Test set size    : 230
Time taken: 5.0539631843566895
The accuracy on the test set is  63.91304347826087

Training set size: 920
Test set size    : 230
Time taken: 4.671295881271362
The accuracy on the test set is  63.04347826086957

Training set size: 920
Test set size    : 230
Time taken: 4.789471387863159
The accuracy on the test set is  66.95652173913044

Training set size: 920
Test set size    : 230
Time taken: 4.864834785461426
The accuracy on the test set is  63.91304347826087

Training set size: 920
Test set size    : 230
Time taken: 4.761426210403442
The accuracy on the test set is  64.34782608695652

The accuracy of the model is  64.43478260869566
